In [1]:
import sys,os
import numpy
import heapq
import time
import json
import pandas as pd
import glob
from sklearn.feature_extraction import DictVectorizer

In [2]:
Train_All = False


In [3]:
training_files = glob.glob('data/training/*')
training_df = pd.DataFrame()
for index, filename in enumerate(training_files):
    with open(filename) as f:
        data = (line.strip() for line in f)
        data_json = "[{0}]".format(','.join(data))
    training_df = pd.concat([training_df, pd.read_json(data_json)], ignore_index=True)
    # for memory reserve
    if Train_All == False and index == 10: 
        break
    print(index)

0
1
2
3
4
5
6
7
8
9


In [4]:
training_df.head()

,author_id,browser,browser_version,category_id,city,cookie_pta,country,date,device_brand,device_marketing,hour,os,os_version,resolution,timestamp,url
0,milk52313,Chrome,20a466bc8756f1f28988bf2e65fd9efeff055d6bb35ab3...,美味食記,,227f747d3020961ca7545dc91166ea893a452f6c965f67...,TW,20150519,,,16,Windows XP,,1024x768,2015-05-19 08:37:53,http://milk52313.pixnet.net/blog/post/423828488
1,qpjj,Other,,美味食記,,71f285727d205b8a7c54a765f3f6cd545dd3288786db7e...,TW,20150327,,,13,Windows 7,,1024x819,2015-03-27 05:56:27,http://qpjj.pixnet.net/blog/post/5103901
2,dreampudding,Chrome Mobile,781faba9107b3b9909066c4e40793fa308ea8a5a5dc6e8...,美味食記,Taipei,5b58b3f61bc8c0a47eb2126c3ac35cd85d4482e1abd1a7...,TW,20150811,hTC,87fde507ac9c015aa48a00cbf8d5678817ed4571c33a16...,20,Android,f374e854df9a0650ff3fb3923e02806cf7f595a53c73f5...,360x640,2015-08-11 12:43:11,http://dreampudding.pixnet.net/blog/post/41912680
3,dreampudding,Chrome Mobile,781faba9107b3b9909066c4e40793fa308ea8a5a5dc6e8...,美味食記,Taipei,5b58b3f61bc8c0a47eb2126c3ac35cd85d4482e1abd1a7...,TW,20150811,hTC,87fde507ac9c015aa48a00cbf8d5678817ed4571c33a16...,20,Android,f374e854df9a0650ff3fb3923e02806cf7f595a53c73f5...,360x640,2015-08-11 12:44:57,http://dreampudding.pixnet.net/blog/post/41912680
4,cline1413,Chrome Mobile,20a466bc8756f1f28988bf2e65fd9efeff055d6bb35ab3...,親子育兒,Taipei,82452818954eefeb18d6fbeb5d8bb4e4205b273172ba3d...,TW,20150418,hTC,344e4fe44f21d600cb04ee706d0ba43131c888d8a08f79...,13,Android,fda195ecd026e4e46842394ef8ee552261bdd0fd03b6e7...,360x640,2015-04-18 05:10:59,http://cline1413.pixnet.net/blog/post/390634931


In [5]:
# encode str col
trainEnc_df = training_df.copy()
encode_dict = {}

M_dict = [ {} for _ in range(12)]
def encode_series(series):
    x = series.astype('category')
    encode_series = x.cat.codes
    dict_df = pd.concat([x.cat.codes, series], axis=1)
    series_dict = dict_df.drop_duplicates().sort_values(0).set_index(0).to_dict()
    
    return encode_series, series_dict
trainEnc_df.browser           ,M_dict[0] = encode_series(trainEnc_df.browser); 
trainEnc_df.browser_version   ,M_dict[1] = encode_series(trainEnc_df.browser_version)
trainEnc_df.category_id       ,M_dict[2] = encode_series(trainEnc_df.category_id)
trainEnc_df.city              ,M_dict[3] = encode_series(trainEnc_df.city)
trainEnc_df.country           ,M_dict[4] = encode_series(trainEnc_df.country)
trainEnc_df.device_brand      ,M_dict[5] = encode_series(trainEnc_df.device_brand)
trainEnc_df.device_marketing  ,M_dict[6] = encode_series(trainEnc_df.device_marketing)
trainEnc_df.os                ,M_dict[7] = encode_series(trainEnc_df.os)
trainEnc_df.os_version        ,M_dict[8] = encode_series(trainEnc_df.os_version)
trainEnc_df.resolution        ,M_dict[9] = encode_series(trainEnc_df.resolution)
trainEnc_df.cookie_pta        ,M_dict[10]= encode_series(trainEnc_df.cookie_pta)
trainEnc_df.author_id         ,M_dict[11]= encode_series(trainEnc_df.author_id)

for i in range(12):
    encode_dict.update(M_dict[i])



In [6]:
training_df.loc[training_df.cookie_pta == 'd5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f71c66c1334']

,author_id,browser,browser_version,category_id,city,cookie_pta,country,date,device_brand,device_marketing,hour,os,os_version,resolution,timestamp,url
911291,imlabor,Chrome,4f5e86ee9a73052d24b5adf4811f2e17cdd12eaa57a39c...,職場甘苦,,d5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f...,TW,20150727,,,10,Windows,,1280x720,2015-07-27 02:16:54,http://imlabor.pixnet.net/blog/post/40548034
1055685,imlabor,Chrome,4f5e86ee9a73052d24b5adf4811f2e17cdd12eaa57a39c...,職場甘苦,,d5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f...,TW,20150727,,,10,Windows,,1280x720,2015-07-27 02:17:22,http://imlabor.pixnet.net/blog/post/35262433
1148030,ifans,Chrome,f8cd71688b6403c74a463d0d7883a5e8fdb842f0c61482...,數位生活,,d5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f...,TW,20150415,,,8,Windows,,1280x720,2015-04-15 00:53:02,http://ifans.pixnet.net/blog/post/196598139
1442670,kellyla1028,Chrome,f8cd71688b6403c74a463d0d7883a5e8fdb842f0c61482...,裝潢設計,Taipei,d5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f...,TW,20150323,,,9,Windows,,1280x720,2015-03-23 01:20:46,http://kellyla1028.pixnet.net/blog/post/44586085
4215094,a0912414333,Chrome,1d18d077ea3991866820b908da17138e2be22951bb8a3c...,醫療保健,,d5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f...,TW,20150623,,,13,Windows,,1280x720,2015-06-23 05:12:07,http://a0912414333.pixnet.net/blog/post/188212776


In [7]:
hidden_units = 10    # how many neurons in the hidden layer
activation = 'sigmoid'  # activation function for hidden layer
l2 = 0.001           # regularization - how much we penalize large parameter values
learning_rate = 0.01  # how big our steps are in gradient descent
epochs = 50          # how many epochs to train for
batch_size = 32      # how many samples to use for each gradient descent update

In [ ]:
def one_hot_encode(series, num):
    encode = list()
    for s in series:
        letter = [0 for _ in range(num)]
        letter[s] = 1
        encode.append(letter)
    return encode
trainEnc1_df.author_id = one_hot_encode(trainEnc_df.author_id, len(encode_dict['author_id']))

In [ ]:
from keras import models, layers, optimizers, regularizers
import keras


train_X = trainEnc_df.copy().drop(['author_id', 'date', 'timestamp', 'url'], axis=1)
train_y = trainEnc_df.author_id
    

# create a sequential model
model = models.Sequential()

# add the hidden layer
model.add ( layers.Dense ( input_dim = 12,
                       units = hidden_units, 
                       activation = activation ) )

# add the output layer
model.add ( layers.Dense ( input_dim = hidden_units,
                       units = 1,
                       activation = 'sigmoid' ) )

# define our loss function and optimizer
model.compile ( loss = 'sparse_categorical_crossentropy',
              # Adam is a kind of gradient descent
              optimizer = optimizers.Adam ( lr = learning_rate ),
              metrics = [ 'accuracy' ] 
                )

# train the parameters
history = model.fit ( train_X, train_y, epochs = 3, batch_size = batch_size )


In [33]:

# evaluate accuracy
train_acc = model.evaluate ( train_X, train_y, batch_size = 32 )
# test_acc = model.evaluate ( test_X, test_y, batch_size = 32 )[1]
print ( 'Training accuracy: %s' % train_acc )
#print ( 'Testing accuracy: %s' % test_acc )

losses = history.history['loss']
#plt.plot ( range ( len ( losses ) ), losses, 'r' )
#plt.show()

4887580/4887580 [==============================] - 78s 16us/step
Training accuracy: 2965982.6844796403


In [22]:
def get_latest_cookie_info(df, in_cookie):
    df_group = df.loc[df.cookie_pta == in_cookie]
    return df_group.loc[df_group.timestamp == df_group.timestamp.max()]
get_latest_cookie_info(training_df, 'd5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f71c66c1334')

,author_id,browser,browser_version,category_id,city,cookie_pta,country,date,device_brand,device_marketing,hour,os,os_version,resolution,timestamp,url
1055685,imlabor,Chrome,4f5e86ee9a73052d24b5adf4811f2e17cdd12eaa57a39c...,職場甘苦,,d5d9ba4a82443aab3a4a0b59ef1aeacd14ea934bfe7d3f...,TW,20150727,,,10,Windows,,1280x720,2015-07-27 02:17:22,http://imlabor.pixnet.net/blog/post/35262433


In [22]:
hidden_units = 10    # how many neurons in the hidden layer
activation = 'sigmoid'  # activation function for hidden layer
l2 = 0.001           # regularization - how much we penalize large parameter values
learning_rate = 0.01  # how big our steps are in gradient descent
epochs = 50          # how many epochs to train for
batch_size = 32      # how many samples to use for each gradient descent update

In [23]:
train_X = trainEnc_df.cookie_pta
train_y = trainEnc_df.author_id
    
from keras import models, layers, optimizers, regularizers
import keras

# create a sequential model
model = models.Sequential()

# add the hidden layer
model.add ( layers.Dense ( input_dim = 1,
                       units = hidden_units, 
                       activation = activation ) )

# add the output layer
model.add ( layers.Dense ( input_dim = hidden_units,
                       units = 1,
                       activation = 'sigmoid' ) )

# define our loss function and optimizer
model.compile ( loss = 'hinge',
              # Adam is a kind of gradient descent
              optimizer = optimizers.Adam ( lr = learning_rate ),
              #metrics = [ 'accuracy' ] 
                )

# train the parameters
history = model.fit ( train_X, train_y, epochs = 5, batch_size = batch_size )

# # evaluate accuracy
# train_acc = model.evaluate ( train_X, train_y, batch_size = 32 )[1]
# test_acc = model.evaluate ( test_X, test_y, batch_size = 32 )[1]
# print ( 'Training accuracy: %s' % train_acc )
# print ( 'Testing accuracy: %s' % test_acc )

# losses = history.history['loss']
# plt.plot ( range ( len ( losses ) ), losses, 'r' )
# plt.show()

Epoch 1/5
10621620/10621620 [==============================] - 305s 29us/step - loss: 1.2461e-04
Epoch 2/5
10621620/10621620 [==============================] - 304s 29us/step - loss: 1.2437e-04
Epoch 3/5
10621620/10621620 [==============================] - 304s 29us/step - loss: 1.2437e-04
Epoch 4/5
10621620/10621620 [==============================] - 304s 29us/step - loss: 1.2437e-04
Epoch 5/5
10621620/10621620 [==============================] - 305s 29us/step - loss: 1.2437e-04


In [24]:

# evaluate accuracy
train_acc = model.evaluate ( train_X, train_y, batch_size = 32 )
# test_acc = model.evaluate ( test_X, test_y, batch_size = 32 )[1]
print ( 'Training accuracy: %s' % train_acc )
#print ( 'Testing accuracy: %s' % test_acc )

losses = history.history['loss']
#plt.plot ( range ( len ( losses ) ), losses, 'r' )
#plt.show()

10621620/10621620 [==============================] - 109s 10us/step
Training accuracy: 0.00012436897573063244


In [31]:
hidden_units = 10    # how many neurons in the hidden layer
activation = 'sigmoid'  # activation function for hidden layer
l2 = 0.001           # regularization - how much we penalize large parameter values
learning_rate = 0.01  # how big our steps are in gradient descent
epochs = 2          # how many epochs to train for
batch_size = 32      # how many samples to use for each gradient descent update

In [32]:
train_X = trainEnc_df.cookie_pta
train_y = trainEnc_df.category_id
    
from keras import models, layers, optimizers, regularizers
import keras

# create a sequential model
model = models.Sequential()

# add the hidden layer
model.add ( layers.Dense ( input_dim = 1,
                       units = hidden_units, 
                       activation = activation ) )

# add the output layer
model.add ( layers.Dense ( input_dim = hidden_units,
                       units = 1,
                       activation = 'sigmoid' ) )

# define our loss function and optimizer
model.compile ( loss = 'hinge',
              # Adam is a kind of gradient descent
              optimizer = optimizers.Adam ( lr = learning_rate ),
              #metrics = [ 'accuracy' ] 
                )

# train the parameters
history = model.fit ( train_X, train_y, epochs = 2, batch_size = batch_size )

Epoch 1/2
10621620/10621620 [==============================] - 338s 32us/step - loss: 0.0013
Epoch 2/2
10621620/10621620 [==============================] - 325s 31us/step - loss: 0.0013


In [34]:

# evaluate accuracy
train_acc = model.evaluate ( train_X, train_y, batch_size = 32 )
# test_acc = model.evaluate ( test_X, test_y, batch_size = 32 )[1]
print ( 'Training accuracy: %s' % train_acc )
#print ( 'Testing accuracy: %s' % test_acc )

losses = history.history['loss']
#plt.plot ( range ( len ( losses ) ), losses, 'r' )
#plt.show()

10621620/10621620 [==============================] - 133s 12us/step
Training accuracy: 0.0012987661027600089


In [ ]:
from sklearn.svm import SVC
def M_findDeviceByCookie(browser, cookie):
    clf = SVC(gamma='auto')
    score = clf.fit(browser, cookie)
    return score, clf
score, M_clf = M_findDeviceByCookie(trainEnc_df.browser, trainEnc_df.cookie_pta)
print(score)

In [29]:
class PIXNET_Recommendation:
    
    def __init__(self,training_data_folder="data/training",test_data_file_path="data/testing.json",target_file_path="tmp/recommendation",k=20):
        self.training_data_folder = training_data_folder
        self.test_data_file_path = test_data_file_path
        self.target_file_path = target_file_path
        self.k = k
    
    def load_data(self):
        data_set ={}
        training_files = os.listdir(self.training_data_folder)
        print(training_files)
        for filename in training_files:
            for line in open("%s/%s"%(self.training_data_folder,filename)):
                data_json = json.loads(line.strip())
                if data_json['cookie_pta'] not in data_set:
                    data_set[data_json['cookie_pta']]={}
                data_set[data_json['cookie_pta']][data_json['author_id']] = 1
        v = DictVectorizer(sparse=True)
        X = v.fit_transform(data_set.values()).T
        author = v.get_feature_names()
        self.data_set =data_set
        self.X = X
        self.author = author
    
    def calculate_item_similarity(self):
        X = self.X
        author = self.author
        distance = sum(X.T.toarray())
        similarity_tmp_file = open("tmp/similarity_top10","w")
        count = 0
        size_per_loop =1000
        while count< X.shape[0]:
            size_max = min(count+size_per_loop,X.shape[0])-count
            similarity = X[count:min(count+size_per_loop,X.shape[0])].dot(X.T).toarray()
            for i in xrange(size_max):
                similarity[i] = similarity[i]/numpy.sqrt(distance[count+i])
            similarity = similarity.T
            for i in xrange(X.shape[0]):
                similarity[i] = numpy.round(similarity[i]/numpy.sqrt(distance[i]),2)
            similarity = similarity.T
            for i in xrange(size_max):
                rank = []
                for j in xrange(X.shape[0]):
                    if similarity[i][j] >0.99:
                        similarity[i][j] = 0
                    rank.append(similarity[i][j])
                top_k_similarity = heapq.nlargest(self.k, xrange(len(rank)), rank.__getitem__)
                data =[]
                for j in xrange(len(top_k_similarity)):
                    data.append("%s,%.2f"%(author[top_k_similarity[j]],rank[top_k_similarity[j]]))
                similarity_tmp_file.write(";".join(data)+"\n")
            del similarity
            count+=size_per_loop
        similarity_tmp_file.close()
    
    def load_recommend_target(self):
        target_file = open(self.test_data_file_path)
        questions = {}
        for i in target_file:
            j =json.loads(i)
            questions[j['cookie_pta']] = 1
        target_file.close()
        self.questions= questions
    
    def recommend_from_item_similarity(self):
        data_set =self.data_set
        questions = self.questions
        author = self.author
        f = open("tmp/similarity_top10")
        count =0
        author_recommend = {}
        for i in f:
            r =i.strip().split(";")
            author_recommend[author[count]] = {}
            for j in r:
                a = j.split(",")
                author_recommend[author[count]][a[0]] = float(a[1])
            count +=1
        f_out = open(self.target_file_path,"w")
        for i in questions.keys():
            log = data_set[i]
            recommend_author = {}
            for author_id in log.keys():
                for r_author in author_recommend[author_id]:
                    if r_author in log:
                        continue
                    if r_author not in recommend_author:
                        recommend_author[r_author] = 0
                    recommend_author[r_author] += author_recommend[author_id][r_author]
            js = {"cookie_pta":i,"author_id":heapq.nlargest(len(recommend_author)/20,recommend_author,key=recommend_author.__getitem__)}
            f_out.write(json.dumps(js)+"\n")
        f_out.close()

    def main(self):
        self.load_data()
        self.calculate_item_similarity()
        self.load_recommend_target()
        self.recommend_from_item_similarity()

In [14]:
recommendation = PIXNET_Recommendation()
recommendation.main()

['part-00022', 'part-00016', 'part-00012', 'part-00020', 'part-00001', 'part-00005', 'part-00006', 'part-00023', 'part-00008', 'part-00015', 'part-00007', 'part-00002', 'part-00003', 'part-00013', 'part-00000', 'part-00017', 'part-00010', 'part-00014', 'part-00004', 'part-00018', 'part-00011', 'part-00021', 'part-00009', 'part-00019']


KeyboardInterrupt: 